In [ ]:
from ratdata import data_manager as dm, process, ingest
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
beta_low = 13
beta_high = 30

for r in dm.RecordingFile.select().where(dm.RecordingFile.condition == 'baseline'):
    filename = Path(r.dirname) / r.filename
    rec = ingest.read_mce_matlab_file(filename)
    if dm.is_recording_rejected(filename.name):
        continue

    fs = int(1/rec.dt)
    
    x = np.mean(rec.electrode_data, 0)
    if dm.is_recording_sliced:
        slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
        start = int(slice.start * fs)
        end = int((slice.start + slice.length) * fs)
        x = x[start:end]

    f, psd = signal.welch(x, fs, nperseg=fs)
    m, b = process.fit_oof(f, psd, 2, 100)
    f[0] = 1e-20
    oof = f ** m * (np.e ** b)
    detrended = psd - oof
    idx = np.logical_and(f >= 10, f <= 35)
    peaks = process.find_peaks(f, detrended, beta_low, beta_high)
    plt.figure(figsize=(12, 6))
    plt.plot(f[idx], detrended[idx])
    plt.title(filename.stem)
    if len(peaks) == 0:
        print('No peaks: %s' % rec.filename)
    else:
        px, py = peaks[0]
        plt.plot(px, py, 'x')
    plt.savefig('plots/peaks/%s_%s.png' % (rec.rat_label, filename.stem), facecolor='white', bbox_inches='tight')
    plt.close()